# Imports

In [1]:
from PIL import Image
import numpy as np
import pandas as pd
from queue import Queue

# Classes

In [2]:
class Coordinate():
    def __init__(self, i, j):
        self.i = i
        self.j = j

    def isEqual(self, that):
        return self.i == that.i and self.j == that.j

    def distanceTo(self, that):
        return math.sqrt(((self.i - that.i) ** 2) + ((self.j - that.j) ** 2))

    def toString(self):
        return str("[" + str(self.i) + ", " + str(self.j) + "]")

    def range(self, fromNode, toNode):
        return (fromNode.i <= self.i and self.i <= toNode.i and fromNode.j <= self.j and self.j <= toNode.j)

    def setIdentifier(self, identifier):
        self.identifier = identifier
    
    def surrounding(self):
        return [
            Coordinate(self.i, self.j + 1), # to EAST
            Coordinate(self.i + 1, self.j), # to SOUTH
            Coordinate(self.i, self.j - 1), # to WEST
            Coordinate(self.i - 1, self.j) # to NORTH
        ]
    
    def getDirection(self, that):
        i = self.i - that.i
        j = self.j - that.j
        if i != 0:
            return 'N' if i > 0 else 'S'
        elif j != 0:
            return 'W' if j > 0 else 'E'
        else:
            return None
        
            

# Utils

In [3]:
def isDigit(n):
    try:
        int(n)
        return True
    except ValueError:
        return  False

# Functions

In [4]:
def imageToMatrix(image_path, width=None, height=None, threshold=128):

    image = Image.open(image_path)
    image_grey = image.convert('L')

    image_binary = image_grey.point(lambda point : point > threshold and 1)

    if (width is not None and height is not None):
        image_binary = image_binary.resize((width, height), Image.NEAREST)

    nparray = np.array(image_binary).astype(int)
    nparray[nparray == 0] = -1
    nparray[nparray == 1] = -2
    return nparray

In [5]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 5)

def printMatrix_color(value):
    if isinstance(value, str) and not isDigit(value):
        if value == 'E':
            color = 'yellow'
        elif value == 'W':
            color = 'green'
        elif value == 'N':
            color = 'cyan'
        elif value == 'S':
            color = 'blue'
        else:
            color = 'white'
    else:
        value = int(value)
        if value == -1:
            color = 'black'
        elif value == -2:
            color = 'white'
        elif value == 0:
            color = 'green'
        else:
            color = 'white'
    return 'background-color: %s' % color

def printMatrix(matrix):
    df = pd.DataFrame(matrix)
    return df.style.applymap(printMatrix_color)

In [6]:
def createBFSMatrix(matrix, c):
    BFSMatrix = np.full(matrix.shape, -1)
    BFSMatrix[c.i, c.j] = 0
    fifo = Queue()
    fifo.put(c)
    while not fifo.empty():
        c = fifo.get()
        value = BFSMatrix[c.i, c.j]
        for s in c.surrounding():
            # if in matrix range and the cell is not a block
            if s.range(Coordinate(0, 0), Coordinate(matrix.shape[0] - 1, matrix.shape[1] - 1)):
                if BFSMatrix[s.i, s.j] == -1 and matrix[s.i, s.j] == -2:
                    BFSMatrix[s.i, s.j] = value + 1
                    fifo.put(s)
    return BFSMatrix

In [7]:
def createDirectionMatrix(BFSMatrix):
    directionMatrix = np.full(matrix.shape, '-1')
    for row, col in np.ndindex(directionMatrix.shape):
        c = Coordinate(row, col)
        if BFSMatrix[c.i, c.j] > 0: 
            minimum = float('inf')
            resultCoord = c
            for s in c.surrounding():
                # if in matrix range and the cell is not a block
                if s.range(Coordinate(0, 0), Coordinate(directionMatrix.shape[0] - 1, directionMatrix.shape[1] - 1)):
                    if BFSMatrix[s.i, s.j] >= 0 and BFSMatrix[s.i, s.j] < minimum:
                        resultCoord = s
                        minimum = BFSMatrix[s.i, s.j]
            letter = c.getDirection(resultCoord)
            if letter is not None:
                directionMatrix[c.i, c.j] = letter
    return directionMatrix

In [8]:
def createBFSDepthMatrix(matrix, c):
    BFSDepthMatrix = np.full((5, matrix.shape[0], matrix.shape[1]), -1)
    for it in range(0, 5):
        BFSDepthMatrix[it, c.i, c.j] = 0
    fifo = Queue()
    for index, s in enumerate(c.surrounding()):
        # if in matrix range and the cell is not a block
        if s.range(Coordinate(0, 0), Coordinate(matrix.shape[0] - 1, matrix.shape[1] - 1)) and matrix[s.i, s.j] == -2:
            s.setIdentifier(index + 1)
            fifo.put(s)
            BFSDepthMatrix[s.identifier, s.i, s.j] = 1
    while not fifo.empty():
        c = fifo.get()
        value = BFSDepthMatrix[c.identifier, c.i, c.j]
        for index, s in enumerate(c.surrounding()):
            # if in matrix range and the cell is not a block
            if s.range(Coordinate(0, 0), Coordinate(matrix.shape[0] - 1, matrix.shape[1] - 1)) and matrix[s.i, s.j] == -2:
                s.setIdentifier(index + 1)
                # if has no value set and not the same direction coming from
                if BFSDepthMatrix[s.identifier, s.i, s.j] == -1 and s.identifier % 4 != ((c.identifier + 2) % 4):
                    fifo.put(s)
                    BFSDepthMatrix[s.identifier, s.i, s.j] = value + 1
    for row, col in np.ndindex(BFSDepthMatrix[0].shape):
        values = BFSDepthMatrix[1:, row, col]
        values = values[values != -1]
        if values.size > 0:
            BFSDepthMatrix[0, row, col] = min(values)
    return BFSDepthMatrix

In [9]:
def editCell(BFSDepthMatrix, c):
    modifiedMatrix = np.full((6, BFSDepthMatrix[0].shape[0], BFSDepthMatrix[0].shape[1]), 0)
    modifiedMatrix[:-1, :, :] = BFSDepthMatrix
    fifo = Queue()
    if BFSDepthMatrix[0, c.i, c.j] > 0:
        for index in range(0, 5):
            # if in matrix range and the cell is not a block
            if c.range(Coordinate(0, 0), Coordinate(modifiedMatrix[0].shape[0] - 1, modifiedMatrix[0].shape[1] - 1)):
                modifiedMatrix[index, c.i, c.j] = -1
        for index, s in enumerate(c.surrounding()):
            # if in matrix range and the cell is not a block
            if s.range(Coordinate(0, 0), Coordinate(modifiedMatrix[0].shape[0] - 1, modifiedMatrix[0].shape[1] - 1)) and modifiedMatrix[0, s.i, s.j] != -1:
                s.setIdentifier(index + 1)
                value = modifiedMatrix[s.identifier, s.i, s.j]
                modifiedMatrix[s.identifier, s.i, s.j] = -1
                # need update
                if modifiedMatrix[0, s.i, s.j] == value:
                    values = modifiedMatrix[1:5, s.i, s.j]
                    filteredValues = values[values != -1]
                    if filteredValues.size > 0:
                        modifiedMatrix[0, s.i, s.j] = min(filteredValues)
                    else:
                        modifiedMatrix[0, s.i, s.j] = -1
                    print(modifiedMatrix[0:5, s.i, s.j])
                    s.setIdentifier(values.tolist().index(modifiedMatrix[0, s.i, s.j]))
                    print(s.identifier, s.i, s.j)
                    fifo.put(s)
    else:
        for index, s in enumerate(c.surrounding()):
            # if in matrix range and the cell is not a block
            if s.range(Coordinate(0, 0), Coordinate(modifiedMatrix[0].shape[0] - 1, modifiedMatrix[0].shape[1] - 1)) and modifiedMatrix[0, s.i, s.j] != -1:
                # Reverse direction
                fromIndex = ((index + 1 + 2) % 4) + 1
                # reachable
                modifiedMatrix[fromIndex, c.i, c.j] = modifiedMatrix[0, s.i, s.j] + 1
                initCoord = Coordinate(c.i, c.j)
                initCoord.setIdentifier(fromIndex)
                fifo.put(initCoord)
    while not fifo.empty():
        c = fifo.get()
        value = modifiedMatrix[c.identifier, c.i, c.j]
        #print(c.identifier, c.i, c.j, value)
        for index, s in enumerate(c.surrounding()):
            # if in matrix range and the cell is not a block
            if s.range(Coordinate(0, 0), Coordinate(modifiedMatrix[0].shape[0] - 1, modifiedMatrix[0].shape[1] - 1)) and matrix[s.i, s.j] == -2:
                s.setIdentifier(index + 1)
                # if has no value set and not the same direction coming from
                if s.identifier % 4 != ((c.identifier + 2) % 4):
                    if (value + 1 < modifiedMatrix[s.identifier, s.i, s.j] or modifiedMatrix[s.identifier, s.i, s.j] == -1):
                        fifo.put(s)
                        modifiedMatrix[s.identifier, s.i, s.j] = value + 1
                        modifiedMatrix[5, s.i, s.j] = modifiedMatrix[5, s.i, s.j] + 1
    for row, col in np.ndindex(modifiedMatrix[0].shape):
        values = modifiedMatrix[1:5, row, col]
        values = values[values != -1]
        if values.size > 0:
            modifiedMatrix[0, row, col] = min(values)
        else:
            modifiedMatrix[0, row, col] = -1
    return modifiedMatrix

# Environment Variables

In [10]:
FILENAME = 'D:/compressed-path-database/resources/labyrinth_5.png'
MATRIX_WIDTH = 30
MATRIX_HEIGHT = 20

# Explanation for the remove block process

First of all create the matrix, it is compound by two values, -2 is accesible and -1 is a block, not accesible

In [11]:
matrix = imageToMatrix(FILENAME, MATRIX_WIDTH, MATRIX_HEIGHT)
printMatrix(matrix)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,-1,-2,-1,-1,-1,-2,-2,-2,-2,-1,-1,-2,-1,-1,-1,-1,-1,-1,-1,-2,-2,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,-1,-2,-2,-2,-1,-1,-1,-1,-2,-1,-2,-2,-2,-2,-2,-2,-2,-1,-1,-1,-2,-2,-2,-2,-2,-2,-2,-2,-2,-1
2,-1,-2,-1,-2,-2,-2,-2,-1,-2,-1,-1,-1,-1,-1,-1,-1,-2,-2,-2,-1,-2,-1,-2,-1,-1,-1,-1,-1,-2,-2
3,-1,-2,-1,-1,-1,-1,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-1,-2,-1,-2,-1,-2,-2,-2,-2,-2,-2,-2,-1
4,-1,-2,-1,-2,-2,-2,-2,-1,-2,-1,-2,-1,-1,-1,-1,-1,-1,-1,-2,-2,-2,-1,-2,-1,-1,-2,-1,-1,-2,-2
5,-1,-2,-2,-2,-1,-1,-2,-2,-2,-2,-2,-1,-2,-2,-2,-2,-2,-1,-2,-1,-1,-1,-2,-2,-1,-2,-1,-1,-1,-1
6,-1,-1,-1,-2,-2,-2,-2,-1,-1,-1,-2,-1,-2,-1,-1,-1,-2,-1,-2,-2,-2,-1,-1,-2,-1,-2,-2,-2,-2,-2
7,-1,-2,-1,-2,-1,-1,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-1,-2,-1,-1,-1,-2,-1,-1,-1,-2,-1,-2
8,-2,-2,-2,-2,-1,-2,-2,-1,-2,-1,-1,-1,-1,-2,-1,-1,-1,-2,-1,-2,-2,-2,-1,-2,-1,-2,-2,-2,-2,-2
9,-2,-1,-2,-1,-1,-2,-1,-1,-2,-1,-1,-2,-1,-2,-1,-2,-1,-2,-1,-1,-1,-1,-1,-2,-2,-2,-1,-1,-2,-1


Creation of the matrix to be used, it is compound of 5 layers of input image. That is, a matrix of dimension of (5 x N x M) where N and M is the dimension of the input image

In [12]:
BFSDepthMatrix = createBFSDepthMatrix(matrix, Coordinate(0, 1))

The first layer is the result for output in which can be transformed into a matrix with the first directions

The other layers provide the information about a specific direction.
- Layer 1: Coming from West only
- Layer 2: Coming from North only
- Layer 3: Coming from East only
- Layer 4: Coming from South only

In [13]:
printMatrix(BFSDepthMatrix[0])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,-1,0,-1,-1,-1,16,15,14,13,-1,-1,26,-1,-1,-1,-1,-1,-1,-1,30,29,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,-1,1,2,3,-1,-1,-1,-1,12,-1,26,25,24,23,22,21,20,-1,-1,-1,28,29,30,31,32,33,34,35,36,-1
2,-1,2,-1,4,5,6,7,-1,11,-1,-1,-1,-1,-1,-1,-1,19,20,21,-1,27,-1,31,-1,-1,-1,-1,-1,37,38
3,-1,3,-1,-1,-1,-1,8,9,10,11,12,13,14,15,16,17,18,-1,22,-1,26,-1,32,33,34,35,36,37,38,-1
4,-1,4,-1,8,9,10,9,-1,11,-1,13,-1,-1,-1,-1,-1,-1,-1,23,24,25,-1,33,-1,-1,36,-1,-1,39,40
5,-1,5,6,7,-1,-1,10,11,12,13,14,-1,20,21,22,23,24,-1,24,-1,-1,-1,34,35,-1,37,-1,-1,-1,-1
6,-1,-1,-1,8,9,10,11,-1,-1,-1,15,-1,19,-1,-1,-1,23,-1,25,26,27,-1,-1,36,-1,38,39,40,41,42
7,-1,13,-1,9,-1,-1,12,13,14,15,16,17,18,19,20,21,22,23,-1,27,-1,-1,-1,37,-1,-1,-1,41,-1,43
8,13,12,11,10,-1,14,13,-1,15,-1,-1,-1,-1,20,-1,-1,-1,24,-1,28,29,30,-1,38,-1,40,41,42,43,44
9,14,-1,12,-1,-1,15,-1,-1,16,-1,-1,25,-1,21,-1,25,-1,25,-1,-1,-1,-1,-1,39,40,39,-1,-1,42,-1


In [14]:
printMatrix(BFSDepthMatrix[1])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,-1,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,-1,-1,2,3,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,29,30,31,32,33,34,35,36,-1
2,-1,-1,-1,-1,5,6,7,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,20,21,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,38
3,-1,-1,-1,-1,-1,-1,-1,9,10,11,12,13,14,15,16,17,18,-1,-1,-1,-1,-1,-1,33,34,35,36,37,38,-1
4,-1,-1,-1,-1,9,10,11,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,24,25,-1,-1,-1,-1,-1,-1,-1,-1,40
5,-1,-1,6,7,-1,-1,-1,11,12,13,14,-1,-1,21,22,23,24,-1,-1,-1,-1,-1,-1,35,-1,-1,-1,-1,-1,-1
6,-1,-1,-1,-1,9,10,11,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,26,27,-1,-1,-1,-1,-1,39,40,41,42
7,-1,-1,-1,-1,-1,-1,-1,13,14,15,16,17,18,19,20,21,22,23,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
8,-1,50,51,22,-1,-1,19,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,29,30,-1,-1,-1,-1,41,42,43,44
9,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,40,41,-1,-1,-1,-1


In [15]:
printMatrix(BFSDepthMatrix[2])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,-1,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,-1,2,-1,4,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,49,-1,31,-1,-1,-1,-1,-1,37,-1
3,-1,3,-1,-1,-1,-1,8,-1,-1,-1,-1,-1,-1,-1,-1,-1,58,-1,22,-1,50,-1,32,-1,-1,-1,-1,-1,38,-1
4,-1,4,-1,-1,-1,-1,9,-1,11,-1,13,-1,-1,-1,-1,-1,-1,-1,23,-1,51,-1,33,-1,-1,36,-1,-1,39,-1
5,-1,5,-1,13,-1,-1,10,-1,12,-1,14,-1,-1,-1,-1,-1,-1,-1,24,-1,-1,-1,34,-1,-1,37,-1,-1,-1,-1
6,-1,-1,-1,8,-1,-1,11,-1,-1,-1,15,-1,29,-1,-1,-1,25,-1,25,-1,-1,-1,-1,36,-1,38,-1,-1,-1,-1
7,-1,-1,-1,9,-1,-1,12,-1,-1,-1,16,-1,30,-1,-1,-1,26,-1,-1,27,-1,-1,-1,37,-1,-1,-1,41,-1,43
8,-1,-1,-1,10,-1,-1,13,-1,15,-1,-1,-1,-1,20,-1,-1,-1,24,-1,28,-1,-1,-1,38,-1,-1,-1,42,-1,44
9,14,-1,12,-1,-1,15,-1,-1,16,-1,-1,-1,-1,21,-1,-1,-1,25,-1,-1,-1,-1,-1,39,-1,45,-1,-1,44,-1


In [16]:
printMatrix(BFSDepthMatrix[3])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,-1,0,-1,-1,-1,16,15,14,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,30,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,-1,19,18,-1,-1,-1,-1,-1,-1,-1,26,25,24,23,22,21,-1,-1,-1,-1,48,47,46,45,44,43,42,41,-1,-1
2,-1,-1,-1,16,15,14,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,57,56,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,-1,-1,-1,-1,-1,-1,16,15,18,17,64,63,62,61,60,59,-1,-1,-1,-1,-1,-1,44,43,42,41,40,39,-1,-1
4,-1,-1,-1,12,11,10,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,53,52,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
5,-1,15,14,-1,-1,-1,14,13,16,15,-1,-1,28,27,26,25,-1,-1,-1,-1,-1,-1,46,-1,-1,-1,-1,-1,-1,-1
6,-1,-1,-1,14,13,12,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,46,45,48,47,-1
7,-1,-1,-1,-1,-1,-1,20,19,18,17,32,31,30,29,28,27,30,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
8,13,12,11,-1,-1,14,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,44,43,44,45,-1
9,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,41,40,-1,-1,-1,-1,-1


In [17]:
printMatrix(BFSDepthMatrix[4])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,-1,0,-1,-1,-1,-1,-1,-1,13,-1,-1,26,-1,-1,-1,-1,-1,-1,-1,-1,29,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,-1,19,-1,17,-1,-1,-1,-1,12,-1,-1,-1,-1,-1,-1,-1,20,-1,-1,-1,28,-1,46,-1,-1,-1,-1,-1,40,-1
2,-1,18,-1,-1,-1,-1,13,-1,11,-1,-1,-1,-1,-1,-1,-1,19,-1,55,-1,27,-1,45,-1,-1,-1,-1,-1,39,-1
3,-1,17,-1,-1,-1,-1,12,-1,14,-1,16,-1,-1,-1,-1,-1,-1,-1,54,-1,26,-1,48,-1,-1,49,-1,-1,-1,-1
4,-1,16,-1,8,-1,-1,13,-1,13,-1,15,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,47,-1,-1,48,-1,-1,-1,-1
5,-1,-1,-1,15,-1,-1,12,-1,-1,-1,18,-1,20,-1,-1,-1,24,-1,-1,-1,-1,-1,-1,45,-1,47,-1,-1,-1,-1
6,-1,-1,-1,24,-1,-1,21,-1,-1,-1,17,-1,19,-1,-1,-1,23,-1,-1,-1,-1,-1,-1,44,-1,-1,-1,44,-1,46
7,-1,13,-1,23,-1,-1,20,-1,22,-1,-1,-1,-1,33,-1,-1,-1,29,-1,-1,-1,-1,-1,43,-1,-1,-1,43,-1,45
8,49,-1,21,-1,-1,18,-1,-1,21,-1,-1,-1,-1,32,-1,-1,-1,28,-1,-1,-1,-1,-1,42,-1,40,-1,-1,43,-1
9,48,-1,20,-1,-1,17,-1,-1,20,-1,-1,25,-1,31,-1,25,-1,27,-1,-1,-1,-1,-1,-1,-1,39,-1,-1,42,-1


For example if we are going to remove a block (2, 10), in the initial node we can set the values from its surroundings
Below is the 3x3 dimension of the surroundings, parsing by 3, they are layers 0, 1, 2, 3, 4, respectively.

In [18]:
def set_color(value):
    color = 'white'
    if str(value).startswith('M_'):
        color = 'cyan'
    return 'background-color: %s' % color

In [19]:
test = np.concatenate((
        BFSDepthMatrix[0, 1:4, 9:12], 
        BFSDepthMatrix[1, 1:4, 9:12],
        BFSDepthMatrix[2, 1:4, 9:12],
        BFSDepthMatrix[3, 1:4, 9:12],
        BFSDepthMatrix[4, 1:4, 9:12]
    ), axis = 1).astype(str)
pd.DataFrame(test).style.applymap(set_color)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,-1,26,25,-1,-1,-1,-1,-1,-1,-1,26,25,-1,-1,-1
1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,11,12,13,11,12,13,-1,-1,-1,17,64,63,-1,16,-1


The first thing is to update the removed block cell, by getting the values from it surroundings. For example, for layer 1, which only accept values from west. Then it checks the minimum value (layer 0) from the west and set, if the from value was -1, then do nothing

In [20]:
testShow = np.copy(test)
testShow[1, 4] = 'M_(-1)'
testShow[1, 7] = 'M_(27)'
testShow[1, 10] = 'M_(-1)'
testShow[1, 13] = 'M_(13)'
pd.DataFrame(testShow).style.applymap(set_color)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,-1,26,25,-1,-1,-1,-1,-1,-1,-1,26,25,-1,-1,-1
1,-1,-1,-1,-1,M_(-1),-1,-1,M_(27),-1,-1,M_(-1),-1,-1,M_(13),-1
2,11,12,13,11,12,13,-1,-1,-1,17,64,63,-1,16,-1


Once the init cell is defined, then from this node can expand to others and will stop the sequence once from this path is much longer.
The reason of picking 4 extra layers to store the data was having already the second possibilities calculated
The idea is to reach a cell, it is only accessible from four directions. So if we store the information of 4 paths to go to a cell, if disappears one of them, it is easily get the second path to replace the disappeared one.

In [21]:
BFSDepthMatrix = createBFSDepthMatrix(matrix, Coordinate(0, 1))
modifiedMatrix = editCell(BFSDepthMatrix, Coordinate(2, 10))
print(modifiedMatrix[1:5, 2, 10])
printMatrix(modifiedMatrix[0])

[13 -1 27 -1]


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,-1,0,-1,-1,-1,16,15,14,13,-1,-1,16,-1,-1,-1,-1,-1,-1,-1,30,29,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,-1,1,2,3,-1,-1,-1,-1,12,-1,14,15,16,17,18,19,20,-1,-1,-1,28,29,30,31,32,33,34,35,36,-1
2,-1,2,-1,4,5,6,7,-1,11,-1,13,-1,-1,-1,-1,-1,19,20,21,-1,27,-1,31,-1,-1,-1,-1,-1,37,38
3,-1,3,-1,-1,-1,-1,8,9,10,11,12,13,14,15,16,17,18,-1,22,-1,26,-1,32,33,34,35,36,37,38,-1
4,-1,4,-1,8,9,10,9,-1,11,-1,13,-1,-1,-1,-1,-1,-1,-1,23,24,25,-1,33,-1,-1,36,-1,-1,39,40
5,-1,5,6,7,-1,-1,10,11,12,13,14,-1,20,21,22,23,24,-1,24,-1,-1,-1,34,35,-1,37,-1,-1,-1,-1
6,-1,-1,-1,8,9,10,11,-1,-1,-1,15,-1,19,-1,-1,-1,23,-1,25,26,27,-1,-1,36,-1,38,39,40,41,42
7,-1,13,-1,9,-1,-1,12,13,14,15,16,17,18,19,20,21,22,23,-1,27,-1,-1,-1,37,-1,-1,-1,41,-1,43
8,13,12,11,10,-1,14,13,-1,15,-1,-1,-1,-1,20,-1,-1,-1,24,-1,28,29,30,-1,38,-1,40,41,42,43,44
9,14,-1,12,-1,-1,15,-1,-1,16,-1,-1,25,-1,21,-1,25,-1,25,-1,-1,-1,-1,-1,39,40,39,-1,-1,42,-1


In [22]:
printMatrix(modifiedMatrix[5])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,1,2,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [23]:
BFSDepthMatrix = createBFSDepthMatrix(matrix, Coordinate(0, 1))
modifiedMatrix = editCell(BFSDepthMatrix, Coordinate(11, 5))
print(modifiedMatrix[1:5, 11, 5])
printMatrix(modifiedMatrix[0])

[27 -1 17 -1]


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,-1,0,-1,-1,-1,16,15,14,13,-1,-1,26,-1,-1,-1,-1,-1,-1,-1,30,29,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,-1,1,2,3,-1,-1,-1,-1,12,-1,26,25,24,23,22,21,20,-1,-1,-1,28,29,30,31,32,33,34,35,36,-1
2,-1,2,-1,4,5,6,7,-1,11,-1,-1,-1,-1,-1,-1,-1,19,20,21,-1,27,-1,31,-1,-1,-1,-1,-1,37,38
3,-1,3,-1,-1,-1,-1,8,9,10,11,12,13,14,15,16,17,18,-1,22,-1,26,-1,32,33,34,35,36,37,38,-1
4,-1,4,-1,8,9,10,9,-1,11,-1,13,-1,-1,-1,-1,-1,-1,-1,23,24,25,-1,33,-1,-1,36,-1,-1,39,40
5,-1,5,6,7,-1,-1,10,11,12,13,14,-1,20,21,22,23,24,-1,24,-1,-1,-1,34,35,-1,37,-1,-1,-1,-1
6,-1,-1,-1,8,9,10,11,-1,-1,-1,15,-1,19,-1,-1,-1,23,-1,25,26,27,-1,-1,36,-1,38,39,40,41,42
7,-1,13,-1,9,-1,-1,12,13,14,15,16,17,18,19,20,21,22,23,-1,27,-1,-1,-1,37,-1,-1,-1,41,-1,43
8,13,12,11,10,-1,14,13,-1,15,-1,-1,-1,-1,20,-1,-1,-1,24,-1,28,29,30,-1,38,-1,40,41,42,43,44
9,14,-1,12,-1,-1,15,-1,-1,16,-1,-1,25,-1,21,-1,25,-1,25,-1,-1,-1,-1,-1,39,40,39,-1,-1,42,-1


In [24]:
printMatrix(modifiedMatrix[5])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [25]:
BFSDepthMatrix = createBFSDepthMatrix(matrix, Coordinate(0, 1))
modifiedMatrix = editCell(BFSDepthMatrix, Coordinate(8, 22))
print(modifiedMatrix[1:5, 8, 22])
printMatrix(modifiedMatrix[0])

[-1 31 -1 39]


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,-1,0,-1,-1,-1,16,15,14,13,-1,-1,26,-1,-1,-1,-1,-1,-1,-1,30,29,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,-1,1,2,3,-1,-1,-1,-1,12,-1,26,25,24,23,22,21,20,-1,-1,-1,28,29,30,31,32,33,34,35,36,-1
2,-1,2,-1,4,5,6,7,-1,11,-1,-1,-1,-1,-1,-1,-1,19,20,21,-1,27,-1,31,-1,-1,-1,-1,-1,37,38
3,-1,3,-1,-1,-1,-1,8,9,10,11,12,13,14,15,16,17,18,-1,22,-1,26,-1,32,33,34,35,36,37,38,-1
4,-1,4,-1,8,9,10,9,-1,11,-1,13,-1,-1,-1,-1,-1,-1,-1,23,24,25,-1,33,-1,-1,36,-1,-1,39,40
5,-1,5,6,7,-1,-1,10,11,12,13,14,-1,20,21,22,23,24,-1,24,-1,-1,-1,34,35,-1,37,-1,-1,-1,-1
6,-1,-1,-1,8,9,10,11,-1,-1,-1,15,-1,19,-1,-1,-1,23,-1,25,26,27,-1,-1,34,-1,38,39,40,41,42
7,-1,13,-1,9,-1,-1,12,13,14,15,16,17,18,19,20,21,22,23,-1,27,-1,-1,-1,33,-1,-1,-1,39,-1,41
8,13,12,11,10,-1,14,13,-1,15,-1,-1,-1,-1,20,-1,-1,-1,24,-1,28,29,30,31,32,-1,36,37,38,39,40
9,14,-1,12,-1,-1,15,-1,-1,16,-1,-1,25,-1,21,-1,25,-1,25,-1,-1,-1,-1,-1,33,34,35,-1,-1,40,-1


In [26]:
printMatrix(modifiedMatrix[5])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,0,1,0,1,0,1,0,0,1,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,1,0,1,0,0,1,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,1,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,1,1,2,1,1
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,2,0,2,0,1,1,2,2,1
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,2,0
